In [4]:
!pip install konlpy
!pip install sentence_transformers
!pip install keybert

In [8]:
# 데이터 불러오기
from NewsAnalyzer.PreProcesser import preprocesser

file_path = "2022-04-12 news data, test.csv"

pc = preprocesser()
df = pc.road_data(file_path)

# 714번째 결측치 제거
df = df.dropna()
df = df.reset_index()

df

,index,카테고리,언론사,기사제목,날짜,본문
0,0,정치,디지털타임스,"김준일 대표, ‘검수완박’ 민주당에 쓴소리…“‘논리적 확증편향’ 빠졌는지 돌아봐야”",2022.04.12.,\n\n\n\n\n‘계곡 살인’ 이은해 사건 거론하며 “‘검수완박’이었다면 이은해 ...
1,1,정치,중앙일보,[단독]윤희숙 父 ‘투기의혹’ 세종시 땅 매각…차익 전액 기부,2022.04.12.,\n\n\n\n\n 지난해 8월 25일 국민권익위원회 조사에서 부동산 관련 불법 ...
2,2,정치,문화일보,전쟁 피해 절박한 젤렌스키… 미지근한 한국 국회,2022.04.12.,"\n\n\n\n\n젤렌스키 우크라 대통령, “무기 요청” 국회 화상연설현장 참석 국..."
3,3,정치,아시아경제,"금태섭 ""文 정권 땐 특수부 키우더니 이젠 검수완박…이러니 내로남불""",2022.04.12.,"\n\n\n\n\n""응징적 차원에서 제도 바꾸면 중립성 해칠 것""금태섭 전 더불어민..."
4,4,정치,머니S,"송영길 ""애도 아니고 5선 정치인 진퇴가 가벼울 수 없다""",2022.04.12.,\n\n\n\n\n송영길 더불어민주당 전 대표가 12일 민주당의 지방선거 승리를 위...
...,...,...,...,...,...,...
1041,1042,세계,한국일보,"친(親)러 도네츠크 군, “우크라 마리우폴 항 장악”",2022.04.12.,"\n\n\n\n\n우크라군, 항만시설ㆍ제철소에서 마지막 저항중 마리우폴 진격한 러시..."
1042,1043,세계,YTN,러시아군 물러났지만...현실화된 우크라발 식량 위기,2022.04.12.,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[앵커]러시아군이 수도 키이우...
1043,1044,세계,뉴스1,"우크라 內 친러 도네츠크 군대 ""마리우폴 장악 임박""",2022.04.12.,\n\n\n\n\n우크라군 이미 몰려 있어…DPR군 전투 강화할 것마리우폴에 있는 ...
1044,1045,세계,연합뉴스,"은혜 갚은 빈살만?…사우디 국부펀드, 쿠슈너에 2조5천억원 투자",2022.04.12.,\n\n\n\n\nNYT 보도…자문단 반대에도 경험 없는 쿠슈너 회사에 거액 투자2...


In [17]:
content = df["본문"][:10]
content

0    \n\n\n\n\n‘계곡 살인’ 이은해 사건 거론하며 “‘검수완박’이었다면 이은해 ...
1    \n\n\n\n\n  지난해 8월 25일 국민권익위원회 조사에서 부동산 관련 불법 ...
2    \n\n\n\n\n젤렌스키 우크라 대통령, “무기 요청” 국회 화상연설현장 참석 국...
3    \n\n\n\n\n"응징적 차원에서 제도 바꾸면 중립성 해칠 것"금태섭 전 더불어민...
4    \n\n\n\n\n송영길 더불어민주당 전 대표가 12일 민주당의 지방선거 승리를 위...
5    \n\n\n\n\n검수완박 강행? 전국 檢 '직 걸고 막겠다' 보완수사 요청도 못해...
6    \n\n\n\n\n"검수완박 후속 조치에 의견 모여? 동의하기 힘들다""분명히 수사...
7    \n\n\n\n\n조국 전 법무부 장관 페이스북. /사진=뉴시스 [파이낸셜뉴스]  ...
8    \n\n\n\n\n"尹 신임 장관 자녀들 표창장 공개해야""허위면 학력 박탈하길"[...
9    \n\n\n\n\n김태흠, 라디오서 정치적 메시지보다 ‘명예 회복’ 우선 강조   ...
Name: 본문, dtype: object

## 키워드 출력_test(Korean KeyBERT)


### 방법 1. 유사한 단어들 출력 - 문서와 가장 유사한 키워드 출력


In [21]:
# 15분 50초 걸림

from konlpy.tag import Okt
from keybert import KeyBERT

def keyBERT(content):
  kw = KeyBERT()
  okt = Okt()
  article = []

  for i in range(len(content)):

    # 결과적으로 불용어까지 제거된 토큰화 목록이 필요함
    # 기사별 명사 추출 -> 기사별로 명사를 추출해야 기사별로 CountVectorizer사용해서 단어사전 만들고 키워드 단어 뽑을 수 있음
    # 임의로 okt.nouns 사용
    words = okt.nouns(content[i])

    # 추출한 명사 리스트에서 빼내기
    tokenized_nouns = ' '.join(word for word in words)

    keywords = kw.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(1, 1), stop_words=None)
    article.append(keywords)

  return article

In [22]:
keyBERT(content)

[[('상황', 0.3089),
  ('이재명', 0.2986),
  ('설명', 0.2862),
  ('국가보안법', 0.2771),
  ('수정', 0.2764)],
 [('전의면', 0.3442),
  ('의원직', 0.319),
  ('여의도', 0.3),
  ('정치인', 0.2972),
  ('통관', 0.2878)],
 [('영등포구', 0.295),
  ('도서관', 0.2722),
  ('대한민국', 0.2626),
  ('의장', 0.2605),
  ('상황', 0.2455)],
 [('문재인', 0.3112),
  ('민주당', 0.3108),
  ('민정', 0.3066),
  ('응징', 0.2917),
  ('청와대', 0.291)],
 [('시선집중', 0.3419),
  ('서울시장', 0.333),
  ('의사표시', 0.3311),
  ('송영길', 0.3303),
  ('정치인', 0.3157)],
 [('검찰총장', 0.3559),
  ('완성', 0.3339),
  ('정치검찰', 0.3323),
  ('징역형', 0.3308),
  ('공화국', 0.329)],
 [('이의신청', 0.4604),
  ('의총', 0.3625),
  ('송치', 0.3413),
  ('민정', 0.3387),
  ('완전', 0.3365)],
 [('집행정지', 0.334),
  ('정경', 0.325),
  ('신청', 0.3219),
  ('윤석열', 0.3216),
  ('황교익', 0.299)],
 [('학교생활기록부', 0.3799),
  ('검찰총장', 0.3022),
  ('청와대', 0.2984),
  ('황교익', 0.2935),
  ('이선영', 0.2855)],
 [('현실정치', 0.4),
  ('동영상', 0.3498),
  ('박정희', 0.3351),
  ('전국동시', 0.3228),
  ('지방선거', 0.3134)]]

## 방법 2-1. 다양한 키워드들을 얻기 위한 두 가지 알고리즘
### [1] Max Sum Similarity

In [24]:
from konlpy.tag import Okt
from keybert import KeyBERT

def keyBERT_mss(content):
  kw = KeyBERT()
  okt = Okt()

  article = []

  for i in range(len(content)):

    # 결과적으로 불용어까지 제거된 토큰화 목록이 필요함
    # 기사별 명사 추출 -> 기사별로 명사를 추출해야 기사별로 CountVectorizer사용해서 단어사전 만들고 키워드 단어 뽑을 수 있음
    # 임의로 okt.nouns 사용
    words = okt.nouns(content[i])

    # 추출한 명사 리스트에서 빼내기
    tokenized_nouns = ' '.join(word for word in words)

    # 
    # nr_cadidates, top_5 인자로 입력 받기
    keywords = kw.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(1, 1), 
                                  stop_words=None, use_maxsum=True, nr_candidates=20, top_n=5)
    article.append(keywords)

  return article

In [26]:
keyBERT_mss(content)

[[('상대방', 0.2406),
  ('이은해', 0.2476),
  ('정서', 0.2547),
  ('의혹', 0.2549),
  ('문재인', 0.2651)],
 [('전액', 0.2431),
  ('매매', 0.2466),
  ('부친', 0.2486),
  ('통화', 0.2591),
  ('여의도', 0.3)],
 [('민간인', 0.2025),
  ('국민', 0.2039),
  ('언어', 0.2067),
  ('상황', 0.2455),
  ('영등포구', 0.295)],
 [('목적', 0.2378),
  ('장관', 0.2404),
  ('문제점', 0.2484),
  ('형사소송법', 0.2696),
  ('청와대', 0.291)],
 [('공천', 0.2398),
  ('질의', 0.2409),
  ('마중', 0.2479),
  ('상황', 0.2607),
  ('의사표시', 0.3311)],
 [('마찬가지', 0.2935),
  ('명의', 0.2968),
  ('정성', 0.3061),
  ('대검찰청', 0.3242),
  ('공화국', 0.329)],
 [('박지훈', 0.293),
  ('청과', 0.3029),
  ('동의', 0.3103),
  ('정치', 0.3136),
  ('실천', 0.3339)],
 [('검증', 0.2282),
  ('동의', 0.2382),
  ('파이낸셜뉴스', 0.2574),
  ('부산대학교', 0.2849),
  ('집행정지', 0.334)],
 [('확인', 0.2125),
  ('황씨', 0.2137),
  ('거의', 0.2228),
  ('증명서', 0.2573),
  ('검찰총장', 0.3022)],
 [('대한민국', 0.2314),
  ('대통령', 0.2513),
  ('유영', 0.2564),
  ('정치', 0.2768),
  ('박정희', 0.3351)]]

### [2] Maximal Marginal Relevance
- MMR은 텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하기 위해 노력합니다.

- diversity가 높을수록 다양성이 높음 

In [27]:
from konlpy.tag import Okt
from keybert import KeyBERT

def keyBERT_mmr(content):
  kw = KeyBERT()
  okt = Okt()

  article = []

  for i in range(len(content)):

    # 결과적으로 불용어까지 제거된 토큰화 목록이 필요함
    # 기사별 명사 추출 -> 기사별로 명사를 추출해야 기사별로 CountVectorizer사용해서 단어사전 만들고 키워드 단어 뽑을 수 있음
    # 임의로 okt.nouns 사용
    words = okt.nouns(content[i])

    # 추출한 명사 리스트에서 빼내기
    tokenized_nouns = ' '.join(word for word in words)

    # 
    # nr_cadidates, top_5 인자로 입력 받기
    keywords = kw.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(1, 1), 
                                   stop_words=None, use_mmr=True, diversity=0.7)

    article.append(keywords)

  return article

In [28]:
keyBERT_mmr(content)

[[('상황', 0.3089),
  ('김혜경', 0.0545),
  ('투입', 0.1191),
  ('마음', 0.1449),
  ('효율', 0.1066)],
 [('전의면', 0.3442),
  ('국민권익위', 0.0728),
  ('시세', 0.1355),
  ('반대', 0.1889),
  ('환원', 0.1378)],
 [('영등포구', 0.295),
  ('후미', 0.1198),
  ('국제사회', 0.0464),
  ('지난달', 0.1341),
  ('카키색', 0.1769)],
 [('문재인', 0.3112),
  ('효과', 0.1427),
  ('대통령령', 0.2201),
  ('범죄수사', 0.0684),
  ('참사', 0.1175)],
 [('시선집중', 0.3419),
  ('출마', 0.1023),
  ('국회', 0.0396),
  ('지난', 0.1177),
  ('대표', 0.1706)],
 [('검찰총장', 0.3559),
  ('가야', 0.054),
  ('하나', 0.0879),
  ('무혐의', 0.2225),
  ('질의', 0.2298)],
 [('이의신청', 0.4604),
  ('눈높이', 0.0686),
  ('부분', 0.1221),
  ('다음', 0.1304),
  ('그게', 0.0448)],
 [('집행정지', 0.334),
  ('혐의', 0.1163),
  ('뇌출혈', 0.0603),
  ('부산대학교', 0.2849),
  ('민씨', 0.1286)],
 [('학교생활기록부', 0.3799),
  ('앞서', 0.0589),
  ('민씨', 0.1631),
  ('혐의', 0.1252),
  ('전문', 0.1737)],
 [('현실정치', 0.4),
  ('계기', 0.0268),
  ('만남', 0.0422),
  ('근대화', 0.1791),
  ('부분', 0.085)]]